In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm_notebook

## Initialization

In [3]:
def initialize_wrd_emb(vocab_size, emb_size):
    WRD_EMB = np.random.randn(vocab_size, emb_size) * 0.01
    return WRD_EMB

def initialize_dense(input_size, output_size):
    W = np.random.randn(output_size, input_size) * 0.01
    b = np.random.randn(output_size, 1) * 0.01
    return W, b

def initialize_parameters(vocab_size, emb_size):
    WRD_EMB = initialize_wrd_emb(vocab_size, emb_size)
    W, b = initialize_dense(emb_size, vocab_size)
    
    parameters = {}
    parameters['WRD_EMB'] = WRD_EMB
    parameters['W'] = W
    parameters['b'] = b
    
    return parameters

## Forward Propagation

In [4]:
def ind_to_word_vecs(inds, parameters):
    """
    inds -- shape: (CBOW_N, number of examples)
    """
    WRD_EMB = parameters['WRD_EMB']
    word_vecs = np.take(WRD_EMB, inds.T, axis=0).T
    
    assert(word_vecs.shape == (WRD_EMB.shape[1], inds.shape[0], inds.shape[1]))
    
    return word_vecs

def mean_(word_vecs):
    word_vecs_mean = np.mean(word_vecs, axis=1)
    word_vecs_mean = word_vecs_mean.reshape(word_vecs.shape[0], -1)
    
    assert(word_vecs_mean.shape == (word_vecs.shape[0], word_vecs.shape[2]))
    
    return word_vecs_mean

def linear_dense(word_vecs_mean, parameters):
    W, b = parameters['W'], parameters['b']
    Z = np.dot(W, word_vecs_mean) + b
    
    assert(Z.shape == (W.shape[0], word_vecs_mean.shape[1]))
    
    return W, b, Z

def softmax(Z):
    softmax_out = np.divide(np.exp(Z), np.sum(np.exp(Z), axis=0, keepdims=True) + 0.001)
    
    assert(softmax_out.shape == Z.shape)

    return softmax_out

def forward_propagation(inds, parameters):
    word_vecs = ind_to_word_vecs(inds, parameters)
    word_vecs_mean = mean_(word_vecs)
    W, b, Z = linear_dense(word_vecs_mean, parameters)
    softmax_out = softmax(Z)
    
    caches = {}
    caches['inds'] = inds
    caches['word_vecs'] = word_vecs
    caches['word_vecs_mean'] = word_vecs_mean
    caches['W'] = W
    caches['b'] = b
    caches['Z'] = Z
    
    return softmax_out, caches

## Cost Function

In [5]:
def cross_entropy(softmax_out, Y):
    m = softmax_out.shape[1]
    cost = -(1 / m) * np.sum(np.sum(Y * np.log(softmax_out + 0.001), axis=0, keepdims=True), axis=1)
    return cost

## Backward Propagation

In [6]:
def softmax_backward(Y, caches):
    Z = caches['Z']
    dL_dZ = Z - Y
    
    assert(dL_dZ.shape == Z.shape)
    
    return dL_dZ

def dense_backward(dL_dZ, caches):
    """
    Z = W * X + b
    """
    W = caches['W']
    b = caches['b']
    word_vecs_mean = caches['word_vecs_mean']
    m = word_vecs_mean.shape[1]
    
    dL_dW = (1 / m) * np.dot(dL_dZ, word_vecs_mean.T)
    dL_db = (1 / m) * np.sum(dL_dZ, axis=1, keepdims=True)
    dL_dword_vecs_mean = np.dot(W.T, dL_dZ)

    assert(W.shape == dL_dW.shape)
    assert(b.shape == dL_db.shape)
    assert(word_vecs_mean.shape == dL_dword_vecs_mean.shape)
    
    return dL_dW, dL_db, dL_dword_vecs_mean

def mean_backward(dL_dword_vecs_mean, caches):
    """
    ele_mean_i = (1 / n) * (ele_1 + ele_2 + ele_n)
    """
    word_vecs = caches['word_vecs']
    CBOW_N = word_vecs.shape[1]
    
#     dL_dword_vecs = (1 / m) * (1 / CBOW_N) * np.ones((dL_dword_vecs_mean.shape[0], CBOW_N)) *\
#         np.sum(dL_dword_vecs_mean, axis=1, keepdims=True)
        
    dL_dword_vecs = (1 / m) * (1 / CBOW_N) * np.sum(dL_dword_vecs_mean, axis=1, keepdims=True)
    assert((word_vecs.shape[0], 1) == dL_dword_vecs.shape)
    
    return dL_dword_vecs

def backward_propagation(Y, caches):
    dL_dZ = softmax_backward(Y, caches)
    dL_dW, dL_db, dL_dword_vecs_mean = dense_backward(dL_dZ, caches)
    dL_dword_vecs = mean_backward(dL_dword_vecs_mean, caches)
    
    gradients = dict()
    gradients['dL_dZ'] = dL_dZ
    gradients['dL_dW'] = dL_dW
    gradients['dL_db'] = dL_db
    gradients['dL_dword_vecs'] = dL_dword_vecs
    
    return gradients

def update_parameters(parameters, caches, gradients, learning_rate):
    CBOW_N = caches['inds'].shape[0]
    vocab_size, emb_size = parameters['WRD_EMB'].shape
    inds = caches['inds']
    WRD_EMB = parameters['WRD_EMB']
    dL_dword_vecs = gradients['dL_dword_vecs']
    m = inds.shape[-1]
    
    
#     updated_WRD_EMD = parameters['WRD_EMB'][inds.T, :] -\
#         learning_rate * gradients['dL_dword_vecs'].T.reshape(1, CBOW_N, -1)
    for i in range(m):
        WRD_EMB[inds[:, i], :] -= dL_dword_vecs.T * learning_rate
    
#     parameters['WRD_EMB'][inds.flatten(), :] = updated_WRD_EMD.reshape(-1, emb_size)
    parameters['W'] -= learning_rate * gradients['dL_dW']
    parameters['b'] -= learning_rate * gradients['dL_db']
    

In [48]:
from sys import getsizeof
import gc


def cbow_model(X, Y, vocab_size, emb_size, learning_rate, epochs, batch_size=256, parameters=None, print_cost=False):
    costs = []
    m = X.shape[1]
    if parameters is None:
        parameters = initialize_parameters(vocab_size, emb_size)

#     batch_inds = list(range(0, m, batch_size))
    for epoch in range(epochs):
#         np.random.shuffle(batch_inds)
        for b, i in enumerate(range(0, m, batch_size)):
            X_batch = X[:, i:i+batch_size]
            Y_batch = Y[:, i:i+batch_size]

            softmax_out, caches = forward_propagation(X_batch, parameters)
            gradients = backward_propagation(Y_batch, caches)
            update_parameters(parameters, caches, gradients, learning_rate)
            cost = cross_entropy(softmax_out, Y_batch)
#             if b % 100 == 0:
#                 print('epoch {}, {}/{} - Cost: {}'.format(epoch, b, 1000, np.squeeze(cost)))
#             gc.collect()
#             del X_batch
#             del Y_batch
#             del caches
#             del gradients
            
            
        costs.append(cost)
        if print_cost and epoch % 1000 == 0:
            print("Cost after epoch {}: {}".format(epoch, np.squeeze(cost)))
        
    return parameters

### Toy data
Sentence: I(0) would(1) like(2) to(3) get(4) a(5) better(6) job(7).  
vocab_size = 8  
```
[0, 2] [1]  
[1, 3] [2]  
[2, 4] [3]  
[3, 5] [4]  
[4, 6] [5]  
[5, 7] [6]
```

### Initialization Test

In [8]:
parameters = initialize_parameters(31295, 100)

### Forward Probagation Test

In [11]:
X[:, 0]

array([ 4520, 16807,  3646, 27552, 22269, 23619, 29791, 11870, 31053,
       27600])

In [12]:
softmax_out, caches = forward_propagation(X[:, 0].reshape(-1, 1), parameters)

In [13]:
np.sum(softmax_out)

0.9999999680468006

### Compute Cost Test

In [14]:
cost = cross_entropy(softmax_out, Y_one_hot[:, 0].reshape(-1, 1))

In [104]:
cost

array([6.87648511])

### Backward Probagation Test

In [25]:
gradients = backward_propagation(Y_one_hot[:, 0].reshape(-1, 1), caches)

### Model Test

In [42]:
cbow_model(X[:, 0].reshape(-1, 1), Y_one_hot[:, 0].reshape(-1, 1), 31295, 100, 0.05, 1, batch_size=256, parameters=None, print_cost=False)

epoch 0, 0/1000 - Cost: 6.876591503818192


{'W': array([[-1.47086901e-02,  2.46776711e-03,  8.18906010e-04, ...,
          7.11362578e-03, -1.51344408e-02,  2.04960263e-02],
        [-8.39211344e-04, -6.51600397e-03,  3.82367199e-03, ...,
          6.86077946e-03, -3.04584365e-02, -5.93998251e-03],
        [ 2.44059705e-03,  1.71246057e-02, -7.00175285e-03, ...,
          4.45684667e-03, -8.81435529e-03, -8.60441116e-03],
        ...,
        [-7.68705416e-03,  9.26354933e-03,  1.38015740e-02, ...,
         -6.52921729e-03,  1.65386648e-02,  4.51150994e-03],
        [-1.99787661e-03, -4.76608309e-03,  1.25434277e-02, ...,
          6.25772980e-03,  1.02281155e-02,  9.27558552e-03],
        [ 5.29647943e-03, -5.31701038e-05, -7.73642428e-03, ...,
          3.97533109e-03, -6.43360645e-03, -1.15651844e-02]]),
 'WRD_EMB': array([[-0.02116959,  0.01432641,  0.01467965, ..., -0.0030714 ,
          0.00436741,  0.00781568],
        [-0.00391822,  0.00905689, -0.00870093, ...,  0.00275129,
         -0.00073205, -0.00215951],
        [

## Stack Overflow data

In [7]:
import pickle

with open('pickles/X.pkl', 'rb') as file:
    X = pickle.load(file)
    
with open('pickles/Y.pkl', 'rb') as file:
    Y = pickle.load(file)
    
assert(X.shape[-1] == Y.shape[-1])

In [8]:
vocab_size = 31295
m = X.shape[-1]
batch_size = 256
emb_size = 100

Y_one_hot = np.zeros((vocab_size, m))
Y_one_hot[Y.flatten(), np.arange(m)] = 1

In [17]:
# ran_sam_inds = np.random.randint(m, size=2)
X_sample, Y_one_hot_sample = X[:, np.arange(10000)], Y_one_hot[:, np.arange(10000)]

In [18]:
print(X_sample.shape, Y_one_hot_sample.shape)

(10, 10000) (31295, 10000)


In [49]:
parameters = cbow_model(X, Y_one_hot, vocab_size, emb_size, 0.05, 2000, batch_size=126, print_cost=True)

NameError: name 'Y_one_hot' is not defined

In [ ]:
WRD_EMB = parameters['WRD_EMB']

# with open('pickles/word_to_id.pkl', 'rb') as file:
#     word_to_id = pickle.load(file)
    
# with open('pickles/id_to_word.pkl', 'rb') as file:
#     id_to_word = pickle.load(file)

In [ ]:
def find_top_n_similar(word, wrd_emb, n=10):
    id_ = word_to_id[word]
    vec_word = wrd_emb[id_, :]
    norm_vec_word = np.linalg.norm(vec_word)
    cos_sim = np.dot(wrd_emb, vec_word.T) / (np.linalg.norm(wrd_emb, axis=1) * norm_vec_word)
    top_n_ind = np.argsort(cos_sim)[-n:][::-1]
    return top_n_ind

In [ ]:
inds = find_top_n_similar('sort', WRD_EMB, 20)

In [ ]:
[id_to_word[id_] for id_ in inds]

In [ ]:
id_to_word[156]

In [ ]:
word_to_id['bfs']

## Toy Data: the quick brown fox jumped over the lazy dog

### CBOW

In [56]:
text_toy = ['the','quick','brown','fox','jumped','over','the','lazy','dog']
id_to_word_toy = dict()
word_to_id_toy = dict()
i = 0
for token in text_toy:
    if token in word_to_id_toy:
        continue
    id_to_word_toy[i] = token
    word_to_id_toy[token] = i
    i += 1
window_size = 1
example_len = 2 * 1 + 1
X_toy = []
Y_toy = []

for i in range(len(text_toy) - example_len + 1):
    X_toy.extend([word_to_id_toy[word] for word in text_toy[i:i+1] + text_toy[i+2:i+3]])
    Y_toy.append(word_to_id_toy[text_toy[i+1]])
        
X_toy = np.array(X_toy)
X_toy = X_toy.reshape(-1, window_size * 2).T
Y_toy = np.array(Y_toy)
Y_toy = Y_toy.reshape(1, -1)

In [57]:
vocab_size = 8
m = X_toy.shape[-1]
emb_size = 6

Y_toy_one_hot = np.zeros((vocab_size, m))
Y_toy_one_hot[Y_toy.flatten(), np.arange(m)] = 1

In [58]:
print(X_toy.shape, Y_toy_one_hot.shape)

(2, 7) (8, 7)


In [ ]:
parameters = cbow_model(X_toy, Y_toy_one_hot, vocab_size, emb_size, 0.0025, 10000000, parameters=parameters, batch_size=126, print_cost=True)

Cost after epoch 0: 1.8594929193940735
Cost after epoch 1000: 1.856402710540856
Cost after epoch 2000: 1.8563665576270325
Cost after epoch 3000: 1.8563304119137587
Cost after epoch 4000: 1.8562942733990428
Cost after epoch 5000: 1.8562581420809632
Cost after epoch 6000: 1.8562220179576037
Cost after epoch 7000: 1.8561859010270512
Cost after epoch 8000: 1.856149791287392
Cost after epoch 9000: 1.8561136887367136
Cost after epoch 10000: 1.8560775933731033
Cost after epoch 11000: 1.85604150519465
Cost after epoch 12000: 1.856005424199442
Cost after epoch 13000: 1.8559693503855694
Cost after epoch 14000: 1.8559332837511224
Cost after epoch 15000: 1.855897224294191
Cost after epoch 16000: 1.8558611720128675
Cost after epoch 17000: 1.855825126905243
Cost after epoch 18000: 1.8557890889694106
Cost after epoch 19000: 1.855753058203463
Cost after epoch 20000: 1.8557170346054948
Cost after epoch 21000: 1.855681018173599
Cost after epoch 22000: 1.8556450089058714
Cost after epoch 23000: 1.8556090

Cost after epoch 191000: 1.8496608166257265
Cost after epoch 192000: 1.8496259980619913
Cost after epoch 193000: 1.8495911863451868
Cost after epoch 194000: 1.8495563814735063
Cost after epoch 195000: 1.8495215834451444
Cost after epoch 196000: 1.8494867922582954
Cost after epoch 197000: 1.8494520079111543
Cost after epoch 198000: 1.8494172304019179
Cost after epoch 199000: 1.8493824597287833
Cost after epoch 200000: 1.849347695889946
Cost after epoch 201000: 1.8493129388836045
Cost after epoch 202000: 1.8492781887079566
Cost after epoch 203000: 1.8492434453612012
Cost after epoch 204000: 1.849208708841538
Cost after epoch 205000: 1.8491739791471662
Cost after epoch 206000: 1.849139256276286
Cost after epoch 207000: 1.8491045402270996
Cost after epoch 208000: 1.849069830997807
Cost after epoch 209000: 1.8490351285866113
Cost after epoch 210000: 1.8490004329917142
Cost after epoch 211000: 1.8489657442113199
Cost after epoch 212000: 1.8489310622436312
Cost after epoch 213000: 1.848896387

Cost after epoch 379000: 1.8432333411493238
Cost after epoch 380000: 1.8431997786549792
Cost after epoch 381000: 1.8431662226775647
Cost after epoch 382000: 1.8431326732153774
Cost after epoch 383000: 1.8430991302667117
Cost after epoch 384000: 1.8430655938298632
Cost after epoch 385000: 1.84303206390313
Cost after epoch 386000: 1.8429985404848077
Cost after epoch 387000: 1.8429650235731958
Cost after epoch 388000: 1.842931513166591
Cost after epoch 389000: 1.842898009263293
Cost after epoch 390000: 1.8428645118616003
Cost after epoch 391000: 1.8428310209598129
Cost after epoch 392000: 1.842797536556231
Cost after epoch 393000: 1.8427640586491558
Cost after epoch 394000: 1.8427305872368882
Cost after epoch 395000: 1.8426971223177304
Cost after epoch 396000: 1.842663663889984
Cost after epoch 397000: 1.8426302119519526
Cost after epoch 398000: 1.8425967665019398
Cost after epoch 399000: 1.8425633275382491
Cost after epoch 400000: 1.842529895059185
Cost after epoch 401000: 1.842496469063

Cost after epoch 567000: 1.8370363146974242
Cost after epoch 568000: 1.8370039479763558
Cost after epoch 569000: 1.8369715874606658
Cost after epoch 570000: 1.8369392331487433
Cost after epoch 571000: 1.8369068850389805
Cost after epoch 572000: 1.8368745431297675
Cost after epoch 573000: 1.8368422074194946
Cost after epoch 574000: 1.8368098779065551
Cost after epoch 575000: 1.8367775545893412
Cost after epoch 576000: 1.8367452374662454
Cost after epoch 577000: 1.8367129265356628
Cost after epoch 578000: 1.836680621795985
Cost after epoch 579000: 1.8366483232456072
Cost after epoch 580000: 1.836616030882924
Cost after epoch 581000: 1.8365837447063313
Cost after epoch 582000: 1.8365514647142254
Cost after epoch 583000: 1.8365191909050016
Cost after epoch 584000: 1.8364869232770573
Cost after epoch 585000: 1.8364546618287891
Cost after epoch 586000: 1.8364224065585955
Cost after epoch 587000: 1.8363901574648742
Cost after epoch 588000: 1.8363579145460251
Cost after epoch 589000: 1.8363256

Cost after epoch 755000: 1.8310587189422995
Cost after epoch 756000: 1.8310274910555633
Cost after epoch 757000: 1.8309962690799284
Cost after epoch 758000: 1.8309650530138746
Cost after epoch 759000: 1.8309338428558806
Cost after epoch 760000: 1.8309026386044258
Cost after epoch 761000: 1.830871440257992
Cost after epoch 762000: 1.8308402478150578
Cost after epoch 763000: 1.8308090612741061
Cost after epoch 764000: 1.8307778806336183
Cost after epoch 765000: 1.8307467058920757
Cost after epoch 766000: 1.830715537047962
Cost after epoch 767000: 1.8306843740997598
Cost after epoch 768000: 1.830653217045952
Cost after epoch 769000: 1.8306220658850223
Cost after epoch 770000: 1.8305909206154567
Cost after epoch 771000: 1.8305597812357386
Cost after epoch 772000: 1.8305286477443536
Cost after epoch 773000: 1.830497520139788
Cost after epoch 774000: 1.8304663984205283
Cost after epoch 775000: 1.83043528258506
Cost after epoch 776000: 1.8304041726318712
Cost after epoch 777000: 1.83037306855

...

Cost after epoch 7018000: 1.7769267072565578
Cost after epoch 7019000: 1.7707414132843722
Cost after epoch 7020000: 1.77244102725598
Cost after epoch 7021000: 1.7783463880305954
Cost after epoch 7022000: 1.7950576908276505
Cost after epoch 7023000: 1.8097709869628908
Cost after epoch 7024000: 1.7886684865941025
Cost after epoch 7025000: 1.7825656128831995
Cost after epoch 7026000: 1.7807669410546636
Cost after epoch 7027000: 1.777361571709744
Cost after epoch 7028000: 1.786856237575849
Cost after epoch 7029000: 1.7822143196110805
Cost after epoch 7030000: 1.7827893156591947
Cost after epoch 7031000: 1.783022604000948
Cost after epoch 7032000: 1.774635339808816
Cost after epoch 7033000: 1.775916373738409
Cost after epoch 7034000: 1.7800453210551463
Cost after epoch 7035000: 1.7983205475002388
Cost after epoch 7036000: 1.7937881491214152
Cost after epoch 7037000: 1.8082991045886476
Cost after epoch 7038000: 1.7952826735320684
Cost after epoch 7039000: 1.7841577346558175
Cost after epoch 

Cost after epoch 7202000: 1.7970170832750132
Cost after epoch 7203000: 1.791918699863234
Cost after epoch 7204000: 1.7875758407835436
Cost after epoch 7205000: 1.785986372249949
Cost after epoch 7206000: 1.78330367684262
Cost after epoch 7207000: 1.7803841446836695
Cost after epoch 7208000: 1.778519936130993
Cost after epoch 7209000: 1.7768918861362764
Cost after epoch 7210000: 1.7740395389507118
Cost after epoch 7211000: 1.7716136685944277
Cost after epoch 7212000: 1.7854083201882525
Cost after epoch 7213000: 1.815611894215835
Cost after epoch 7214000: 1.7929059106982979
Cost after epoch 7215000: 1.7736556266905423
Cost after epoch 7216000: 1.7673716667718018
Cost after epoch 7217000: 1.7803012923704802
Cost after epoch 7218000: 1.7816087214071943
Cost after epoch 7219000: 1.7778246074019424
Cost after epoch 7220000: 1.771760615796639
Cost after epoch 7221000: 1.771168309599841
Cost after epoch 7222000: 1.7781489292086328
Cost after epoch 7223000: 1.78517574343608
Cost after epoch 722

Cost after epoch 7386000: 1.788615287094395
Cost after epoch 7387000: 1.7754423769227166
Cost after epoch 7388000: 1.768703994847547
Cost after epoch 7389000: 1.7731721582572881
Cost after epoch 7390000: 1.788484246238078
Cost after epoch 7391000: 1.8166102782295974
Cost after epoch 7392000: 1.781982699681048
Cost after epoch 7393000: 1.7696845970454003
Cost after epoch 7394000: 1.7676865324752962
Cost after epoch 7395000: 1.7681759865268554
Cost after epoch 7396000: 1.7692731354544093
Cost after epoch 7397000: 1.7910335695156738
Cost after epoch 7398000: 1.7957760662527698
Cost after epoch 7399000: 1.795906332841976
Cost after epoch 7400000: 1.7781963222232906
Cost after epoch 7401000: 1.7767381349270657
Cost after epoch 7402000: 1.779738526323301
Cost after epoch 7403000: 1.7908898764519303
Cost after epoch 7404000: 1.7839691144795329
Cost after epoch 7405000: 1.7639130490465897
Cost after epoch 7406000: 1.7614359627424352
Cost after epoch 7407000: 1.7736115338268459
Cost after epoch

Cost after epoch 7570000: 1.7968816054638792
Cost after epoch 7571000: 1.78307141832111
Cost after epoch 7572000: 1.7792001629200236
Cost after epoch 7573000: 1.7800554054508297
Cost after epoch 7574000: 1.7905742470010575
Cost after epoch 7575000: 1.7911663163992264
Cost after epoch 7576000: 1.776912404345078
Cost after epoch 7577000: 1.764785956561763
Cost after epoch 7578000: 1.7642717691191971
Cost after epoch 7579000: 1.7773517375341241
Cost after epoch 7580000: 1.7947657108840884
Cost after epoch 7581000: 1.779127310445321
Cost after epoch 7582000: 1.7827446769812443
Cost after epoch 7583000: 1.775938666658646
Cost after epoch 7584000: 1.7828500573614767
Cost after epoch 7585000: 1.7954801201295505
Cost after epoch 7586000: 1.78878181872997
Cost after epoch 7587000: 1.7824962122555383
Cost after epoch 7588000: 1.7880324519231847
Cost after epoch 7589000: 1.7714102985627165
Cost after epoch 7590000: 1.7669423100860995
Cost after epoch 7591000: 1.7721008846217066
Cost after epoch 7

Cost after epoch 7754000: 1.8128354062891603
Cost after epoch 7755000: 1.7828622464487422
Cost after epoch 7756000: 1.7751616697970718
Cost after epoch 7757000: 1.7777198511208936
Cost after epoch 7758000: 1.7996253470806993
Cost after epoch 7759000: 1.7952615522598523
Cost after epoch 7760000: 1.7848139025122123
Cost after epoch 7761000: 1.7845056784446096
Cost after epoch 7762000: 1.785928098915033
Cost after epoch 7763000: 1.8039274962950422
Cost after epoch 7764000: 1.7916890293667322
Cost after epoch 7765000: 1.80636787689749
Cost after epoch 7766000: 1.80960830636208
Cost after epoch 7767000: 1.778500730885023
Cost after epoch 7768000: 1.773097248150479
Cost after epoch 7769000: 1.7851445521378628
Cost after epoch 7770000: 1.7843469361466626
Cost after epoch 7771000: 1.7830840378848873
Cost after epoch 7772000: 1.7882372489983065
Cost after epoch 7773000: 1.7791530255443369
Cost after epoch 7774000: 1.7751998400373685
Cost after epoch 7775000: 1.774966914630968
Cost after epoch 7

In [61]:
WRD_EMB = parameters['WRD_EMB']

In [62]:
def find_top_n_similar(word, wrd_emb, n=10):
    id_ = word_to_id_toy[word]
    vec_word = wrd_emb[id_, :]
    norm_vec_word = np.linalg.norm(vec_word)
    cos_sim = np.dot(wrd_emb, vec_word.T) / (np.linalg.norm(wrd_emb, axis=1) * norm_vec_word)
    top_n_ind = np.argsort(cos_sim)[-n:][::-1]
    return [id_to_word_toy[id_] for id_ in top_n_ind]

In [66]:
find_top_n_similar('lazy', WRD_EMB)

['lazy', 'the', 'quick', 'jumped', 'brown', 'over', 'fox', 'dog']